### Cloud Snapshots<br>

***

<br>

Snapshot is a great way to back up data any running or stopped instances<br>
from unexpected data loss<br>

Snapshot is a great strategy for use in a back up plan for any and all instances<br>
<br>

**Snapshots:**<br>

- Great way to backup and restore of persistent disks<br>
- Global resources<br>
- Support for zonal and regional PDs<br>
- Incremental and automatically compressed<br>
- Snapshots are stored in Cloud storage<br>
if you store your snapshot at same region, there's no charge<br>
but if you access it from another region, there's a network charge<br>
- Stored in regional or multi-regional location<br>
there are multiple copies stored across the regions<br>
you cannot change the location of existing snapshot<br>
once snapshots been taken, you can use it to create a new disk in any region/zone<br>
<br>

**Creating Snapshots:**<br>

Disk1 -> Snapshot1(full snapshot) -> (incremental) Snapshot2(only new/modified data from Snapshot1)<br>
-> Snapshot3(will contain any changed data from snapshot2), it holds references to snapshot1 and 2 for unchanged data<br>
<br>

**Deleting Snapshots:**<br>

The system marks deleted in the system, and if there is no dependent snapshot, it'll be deleted outright<br>
if it has a dependent, the previous reference will point to the next<br> 
and any necessary changes of the info will move to the next snapshot, increasing its size<br>

This won't completely delete the snapshot<br>
but if you want to make sure, then delete all the snapshots<br>
<br>

**Scheduled Snapshots:**<br>

you don't have to worry about manually creating a snapshot<br>
this is the best practice for backups and must be in same region<br>

create a snapshot schedule and attach to a persisting disks<br>
or<br>
create a new persisting disk with a snapshot schedule<br>

options: setting up a retention policy<br>
options: source disk deletion rule - controls what happens to the snapshots if the source disk is deleted<br>

some downsides of snapshots:<br>
pd can have 1 snapshot scheduled attached at a time<br>
also can't delete a snapshot schedule if it's attached to a disk<br>
you must detach the schedule all disks and delete the schedule<br>
after you create the schedule, you cannot edit it<br>
to update the schedule, you must delete it and create a new one<br>
<br>

**Managing Snapshots:**<br>

- 1 snapshot = 10 min<br>
- create regular schedule<br>
- Eliminate excessive snapshots by creating an image and reusing it<br>
- set schedule to off-peak hours<br>
- Windows - create Volume Shadow Copy Service(VSS) snapshots<br>
<br>

**Demo**<br>

**Creating snapshot**<br>

Check the VPC network and see the firewall rule for ssh is placed in default network<br>
Google Cloud -> Compute Engine -> VM instances<br>
Create the instance via command line this time<br>
> gcloud compute instances create instance1 --zone=us-east1-b --machine-type=e2-micro --subnet=default --image=debian-10-buster-v20201112 --image-project=debian-cloud --boot-disk-size=10GB --boot-disk-type=pd-standard<br>
> gcloud compute ssh --zone "us-east1-b" "instance1"<br>

enter the passphrase<br>

verify the name of the instance<br>
> hostname<br>
> sudo nano fileoftext.txt<br>
> ls -al<br>

Compute Engine -> Disks<br>
pick the instance disk and in dropdown menu choose "create snapshot"<br>

Another way to do it:<br>
Compute Engine -> Snapshots -> create a new snapshot<br>
name: snapshot1<br>
source disk: instance1<br>
location: regional<br>
select location: us-east1<br>
label: key: env, value: testing<br>
<br>

This time we'll create an instance using the snapshot<br>
Compute Engine -> Instances -> Create a new instance<br>
name: instance2<br>
label: key: env, value: testing<br>
region: us-east1, us-east1-b<br>
machine type: e2-micro<br>
boot disk: snapshot > instance1:snapshot<br>
<br>

SSH into instance2<br>
> hostname<br>
> ls -al<br>

the file of fileoftxt.txt is restored there.<br>
<br>

**Creating snapshot schedules**<br>

Google Cloud -> Compute Engine -> Snapshots -> Create Snapshot<br>
name: diskschedule<br>
region: us-east1, us-east1-b<br>
snapshot location: regional: us-east1<br>
schedule freqeuncy: daily<br>
start time (UTC): 6-7 (1AM EST)<br>
auto delete after: 14 days<br>
deletion rule: keep snapshots<br>
enable VSS: no<br>
label: key: env, value: testing<br>
<br>

Now we need to attach it to a disk<br>
Go to Compute Engine -> Disks -> click on instance1 -> edit<br>
snapshot schedule > diskschedule<br>
<br>

Creating snapshot schedule on command line:<br>

> gcloud compute resource-policies create snapshot-schedule diskschedule --region=us-east1 --max-retention-days=14 --on-source-disk-delete=apply-retention-policy --daily-schedule --start-time=06:00 --storage-location=us-east1<br>

insufficient permission<br>

> gcloud auth login<br>
> gcloud compute disks add-resource-policies instance1 --resource-policies diskschedule --zone us-east1-b<br>

this attaches the diskschedules to the disk<br>
<br>